In [ ]:
!pip install nltk
!pip install sentencepiece

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import sys
sys.path.append("../")
sys.path.append("../src/")
from src.camembert.camembert_preprocessor import CamembertPreprocessor
from src.camembert.camembert_trainer import CamembertTrainer

In [ ]:
preprocessor = CamembertPreprocessor()
trainer = CamembertTrainer()

In [ ]:
import pyarrow.parquet as pq
import s3fs


fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'}
)
# Load data
df = pq.read_table(
    "projet-ape/extractions/20240124_sirene4.parquet", filesystem=fs
).to_pandas()

df.head()


In [ ]:
# Preprocess data
df_train, df_test = preprocessor.preprocess(
    df=df,
    y="apet_finale",
    text_feature="activ_pr_lib_et",
    categorical_features=["activ_nat_et", "activ_surf_et"],
)

In [ ]:




    print(f"*** Done! Preprocessing lasted {round((time.time() - t)/60,1)} minutes.\n")

    # Run training of the model
    print("*** 2- Training the model...\n")
    t = time.time()
    model = trainer.train(df_train, Y, text_feature, categorical_features, params)
    print(f"*** Done! Training lasted {round((time.time() - t)/60,1)} minutes.\n")

    if model_type == "fasttext":
        fasttext_model_path = run_name + ".bin"
        model.save_model(fasttext_model_path)

        artifacts = {
            "fasttext_model_path": fasttext_model_path,
            "train_data": "train_text.txt",
        }

        inference_params = {
            "k": 1,
        }
        # Infer the signature including parameters
        signature = mlflow.models.infer_signature(
            params=inference_params,
        )

        mlflow.pyfunc.log_model(
            artifact_path=run_name,
            code_path=["src/fasttext_classifier/", "src/base/", "src/utils/"],
            python_model=FastTextWrapper(text_feature, categorical_features),
            artifacts=artifacts,
            signature=signature,
        )
    elif model_type == "pytorch":
        mlflow.pytorch.log_model(pytorch_model=model, artifact_path=run_name)
    else:
        raise KeyError("Model type is not valid.")

    # Log parameters
    for param_name, param_value in params.items():
        mlflow.log_param(param_name, param_value)
    mlflow.log_param("features", categorical_features)
    mlflow.log_param("Y", Y)

    # Evaluation
    print("*** 3- Evaluating the model...\n")
    t = time.time()
    if model_type == "fasttext":
        evaluator = framework_classes["evaluator"](model)
    elif model_type == "pytorch":
        evaluator = framework_classes["evaluator"](model, trainer.tokenizer)
    else:
        raise KeyError("Model type is not valid.")

    accuracies = evaluator.evaluate(df_test, Y, text_feature, categorical_features, 5)

    # Log metrics
    for metric, value in accuracies.items():
        mlflow.log_metric(metric, value)

    print(f"*** Done! Evaluation lasted {round((time.time() - t)/60,1)} minutes.\n")

    # Tests
    print("*** 4- Performing standard tests...\n")
    t = time.time()
    with open("src/tests/tests.yaml", "r", encoding="utf-8") as stream:
        tests = yaml.safe_load(stream)
    for case in tests.keys():
        run_test(tests[case], preprocessor, evaluator)

    print(f"*** Done! Tests lasted {round((time.time() - t)/60,1)} minutes.\n")
